## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load tensorflow

In [0]:
import pandas as pd
import tensorflow as tf

### Collect Data

In [0]:

data_prices_df=  pd.read_csv('/content/drive/My Drive/collab_ml_files/prices.csv')

### Check all columns in the dataset

In [60]:
data_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data_prices_df=data_prices_df.drop(['date','symbol'],axis=1)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_prices=data_prices_df.head(1000)

In [63]:
data_prices['volume']=data_prices['volume'] / 1000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
data_prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
#splitting dataframe into dependent and independent columns
Y=data_prices['volume']
X=data_prices.drop(['volume'],axis=1)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [39]:
type(x_train)

pandas.core.frame.DataFrame

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
x_train=x_train.to_numpy('float32')
x_test=x_test.to_numpy('float32')
y_train=y_train.to_numpy('float32')
y_test=y_test.to_numpy('float32')

In [145]:
type(x_train)

numpy.ndarray

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
import numpy as np  
#x_train_n = normalize(np.vstack([x_train]), norm='l2', axis=1)
#x_test_n = normalize(np.vstack([x_test]), norm='l2', axis=1)
transformer = Normalizer().fit(x_train)
x_train_new = transformer.transform(x_train)

transformer_test = Normalizer().fit(x_test)
x_test_new = transformer_test.transform(x_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable(tf.random_normal(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
y_pred = tf.add(tf.matmul(x_train_new,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [0]:
loss_fun = tf.reduce_mean(tf.square(y_pred-y_train),name='MSE')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss_fun)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

Observe the training loss at every iteration

In [0]:
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x_input')
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y_input')

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [153]:

for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss_fun],feed_dict={x:x_train_new, y_:y_train})
    
    print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  190.95073
Training loss at step:  1  is  187.2394
Training loss at step:  2  is  184.3653
Training loss at step:  3  is  182.13962
Training loss at step:  4  is  180.416
Training loss at step:  5  is  179.08124
Training loss at step:  6  is  178.0476
Training loss at step:  7  is  177.24715
Training loss at step:  8  is  176.62724
Training loss at step:  9  is  176.14723
Training loss at step:  10  is  175.77548
Training loss at step:  11  is  175.4876
Training loss at step:  12  is  175.26465
Training loss at step:  13  is  175.092
Training loss at step:  14  is  174.9583
Training loss at step:  15  is  174.85478
Training loss at step:  16  is  174.7746
Training loss at step:  17  is  174.71251
Training loss at step:  18  is  174.66444
Training loss at step:  19  is  174.6272
Training loss at step:  20  is  174.59837
Training loss at step:  21  is  174.57602
Training loss at step:  22  is  174.55875
Training loss at step:  23  is  174.54533
Training loss

In [0]:
sess.close()

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [157]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss_fun],feed_dict={x:x_train_new, y_:y_train})
    if epoch % 5 == 0:
      print ('Training loss at step: ', epoch, ' is ', train_loss)
     

Training loss at step:  0  is  195.40076
Training loss at step:  5  is  180.32065
Training loss at step:  10  is  176.12068
Training loss at step:  15  is  174.95094
Training loss at step:  20  is  174.62517
Training loss at step:  25  is  174.53442
Training loss at step:  30  is  174.50916
Training loss at step:  35  is  174.50212
Training loss at step:  40  is  174.50015
Training loss at step:  45  is  174.49962
Training loss at step:  50  is  174.49945
Training loss at step:  55  is  174.49942
Training loss at step:  60  is  174.4994
Training loss at step:  65  is  174.49939
Training loss at step:  70  is  174.4994
Training loss at step:  75  is  174.4994
Training loss at step:  80  is  174.49939
Training loss at step:  85  is  174.4994
Training loss at step:  90  is  174.4994
Training loss at step:  95  is  174.4994


### Get the shapes and values of W and b

In [158]:
print ('W: \n',sess.run(W))
print ('b: \n',sess.run(b))

W: 
 [[ 2.1801274 ]
 [ 2.8023412 ]
 [-0.29157823]
 [ 1.434224  ]]
b: 
 [2.2859507]


In [0]:
sess.close()

### Model Prediction on 1st Examples in Test Dataset

In [119]:
y_test_pred = tf.add(tf.matmul(x_test_new,W),b,name='output')
y_test_pred

<tf.Tensor 'output_5:0' shape=(200, 1) dtype=float32>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [207]:
data_iris = pd.read_csv('/content/drive/My Drive/collab_ml_files/11_Iris.csv')

data_iris=data_iris.drop(['Id'],axis=1)
data_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = data_iris.iloc[:,0:4].values
y = data_iris.iloc[:,4].values


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas

In [0]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
#Y = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
from keras.models import Sequential
from keras.layers import Dense
# create model
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [220]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
model.fit(X_train,Y_train, epochs=150, batch_size=10)

Epoch 1/150
100/100 [==============================] - 0s 4ms/step - loss: 1.6026 - acc: 0.6700
Epoch 2/150
100/100 [==============================] - 0s 286us/step - loss: 0.6518 - acc: 0.7000
Epoch 3/150
100/100 [==============================] - 0s 268us/step - loss: 0.6112 - acc: 0.7000
Epoch 4/150
100/100 [==============================] - 0s 276us/step - loss: 0.6266 - acc: 0.6900
Epoch 5/150
100/100 [==============================] - 0s 289us/step - loss: 0.5825 - acc: 0.6900
Epoch 6/150
100/100 [==============================] - 0s 295us/step - loss: 0.5398 - acc: 0.7900
Epoch 7/150
100/100 [==============================] - 0s 312us/step - loss: 0.5521 - acc: 0.7000
Epoch 8/150
100/100 [==============================] - 0s 265us/step - loss: 0.4997 - acc: 0.8000
Epoch 9/150
100/100 [==============================] - 0s 271us/step - loss: 0.5226 - acc: 0.7400
Epoch 10/150
100/100 [==============================] - 0s 288us/step - loss: 0.5242 - acc: 0.7000
Epoch 11/150
100/100 

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.174486). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 0s 301us/step - loss: 0.3004 - acc: 0.9300
Epoch 45/150
100/100 [==============================] - 0s 319us/step - loss: 0.2987 - acc: 0.9300
Epoch 46/150
100/100 [==============================] - 0s 449us/step - loss: 0.2857 - acc: 0.9600
Epoch 47/150
100/100 [==============================] - 0s 342us/step - loss: 0.2844 - acc: 0.9100
Epoch 48/150
100/100 [==============================] - 0s 278us/step - loss: 0.2808 - acc: 0.9700
Epoch 49/150
100/100 [==============================] - 0s 288us/step - loss: 0.2809 - acc: 0.9300
Epoch 50/150
100/100 [==============================] - 0s 269us/step - loss: 0.2817 - acc: 0.9300
Epoch 51/150
100/100 [==============================] - 0s 295us/step - loss: 0.2705 - acc: 0.9500
Epoch 52/150
100/100 [==============================] - 0s 327us/step - loss: 0.2741 - acc: 0.9400
Epoch 53/150
100/100 [==============================] - 0s 305us/step - loss: 0.2854 - acc: 0.9300
Epoch 54/150
100/100 [=

### Model Prediction

In [222]:
scores = model.evaluate(X_test, Y_test)
#print("model score: " , scores*100)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#Y_predict=model.predict_classes(X_test)
#metrics.accuracy_score(Y_test,Y_predict)

50/50 [==============================] - 0s 193us/step
acc: 98.00%


### Save the Model

In [0]:
import pickle
pickle.dump(model, open("model.pkl","wb" , ))
#loading a model from a file called model.pkl
model = pickle.load(open("model.pkl", "rb"))
##"rb" : this means read mode

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?